<img src="../Pics/MLSb-T.png" width="160">
<br><br>
<center><u><H1>1D ConvNet on Sentiment Analysis</H1></u></center>

In [1]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.log_device_placement = True
sess = tf.Session(config=config)
set_session(sess)

Using TensorFlow backend.


In [2]:
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, Embedding, LSTM, CuDNNLSTM, Dropout, Flatten, Input
from keras.datasets import imdb

In [3]:
num_words = 20000

In [4]:
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=num_words)

In [5]:
print(len(X_train), 'train_data')
print(len(X_test), 'test_data')

25000 train_data
25000 test_data


In [6]:
max_len = 256
embedding_size = 50
batch_size = 128

## Sequences per time

In [7]:
print(X_train[2])

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65, 458, 4468, 66, 3941, 4, 173, 36, 256, 5, 25, 100, 43, 838, 112, 50, 670, 2, 9, 35, 480, 284, 5, 150, 4, 172, 112, 167, 2, 336, 385, 39, 4, 172, 4536, 1111, 17, 546, 38, 13, 447, 4, 192, 50, 16, 6, 147, 2025, 19, 14, 22, 4, 1920, 4613, 469, 4, 22, 71, 87, 12, 16, 43, 530, 38, 76, 15, 13, 1247, 4, 22, 17, 515, 17, 12, 16, 626, 18, 19193, 5, 62, 386, 12, 8, 316, 8, 106, 5, 4, 2223, 5244, 16, 480, 66, 3785, 33, 4, 130, 12, 16, 38, 619, 5, 25, 124, 51, 36, 135, 48, 25, 1415, 33, 6, 22, 12, 215, 28, 77, 52, 5, 14, 407, 16, 82, 10311, 8, 4, 107, 117, 5952, 15, 256, 4, 2, 7, 3766, 5, 723, 36, 71, 43, 530, 476, 26, 400, 317, 46, 7, 4, 12118, 1029, 13, 104, 88, 4, 381, 15, 297, 98, 32, 2071, 56, 26, 141, 6, 194, 7486, 18, 4, 226, 22, 21, 134, 476, 26, 480, 5, 144, 30, 5535, 18, 51, 36, 28, 224, 92, 25, 104, 4, 226, 65, 16, 38, 1334, 88, 12, 16, 283, 5, 16, 4472, 113, 103, 32, 15, 16, 5345, 19, 178, 32]


In [8]:
len(X_train[1])

189

In [10]:
pad  =  'post' #'pre'
#Convert our lists to equal length sequences
X_train_pad = pad_sequences(X_train, maxlen=max_len, padding=pad, truncating=pad)
X_test_pad = pad_sequences(X_test, maxlen=max_len, padding=pad, truncating=pad)

In [11]:
X_train_pad.shape

(25000, 256)

In [12]:
X_train_pad[0]

array([    1,    14,    22,    16,    43,   530,   973,  1622,  1385,
          65,   458,  4468,    66,  3941,     4,   173,    36,   256,
           5,    25,   100,    43,   838,   112,    50,   670,     2,
           9,    35,   480,   284,     5,   150,     4,   172,   112,
         167,     2,   336,   385,    39,     4,   172,  4536,  1111,
          17,   546,    38,    13,   447,     4,   192,    50,    16,
           6,   147,  2025,    19,    14,    22,     4,  1920,  4613,
         469,     4,    22,    71,    87,    12,    16,    43,   530,
          38,    76,    15,    13,  1247,     4,    22,    17,   515,
          17,    12,    16,   626,    18, 19193,     5,    62,   386,
          12,     8,   316,     8,   106,     5,     4,  2223,  5244,
          16,   480,    66,  3785,    33,     4,   130,    12,    16,
          38,   619,     5,    25,   124,    51,    36,   135,    48,
          25,  1415,    33,     6,    22,    12,   215,    28,    77,
          52,     5,

## Creating the model (Model 0):

In [ ]:
model0 = Sequential()

In [ ]:
model0.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_len,
                    name='layer_embedding'))

In [ ]:
model0.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))

In [ ]:
model0.add(MaxPooling1D(pool_size=2))

In [ ]:
model0.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))

In [ ]:
model0.add(MaxPooling1D(pool_size=2))

In [ ]:
model0.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))

In [ ]:
model0.add(MaxPooling1D(pool_size=2))

In [ ]:
model0.add(Dropout(0.5))

In [ ]:
model0.add(Flatten())

In [ ]:
model0.add(Dense(250, activation='relu'))

In [ ]:
model0.add(Dense(1, activation='sigmoid', name='classification'))

## Compiling the model:

In [ ]:
model0.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model0.summary()

## Training

In [ ]:
%%time
model0.fit(X_train_pad, y_train, validation_split=0.05, epochs=10, batch_size=batch_size)

## Testing

In [ ]:
%%time
eval_ = model0.evaluate(X_test_pad, y_test)

In [ ]:
print("Loss: {0:.5}".format(eval_[0]))
print("Accuracy: {0:.2%}".format(eval_[1]))

## Model 1:

In [ ]:
model1 = Sequential()

In [ ]:
model1.add(Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_len,
                    name='layer_embedding'))

In [ ]:
model1.add(Conv1D(filters=128, kernel_size=5, padding='same', activation='relu'))

In [ ]:
model1.add(MaxPooling1D(pool_size=3))

In [ ]:
model1.add(CuDNNLSTM(128))

In [ ]:
model1.add(Dropout(0.5))

In [ ]:
model1.add(Dense(1, activation='sigmoid', name='classification'))

## Compiling the model:

In [ ]:
model1.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
model1.summary()

## Training the model:

In [ ]:
%%time
model1.fit(X_train_pad, y_train, validation_split=0.05, epochs=10, batch_size=batch_size)

## Testing the model:

In [ ]:
%%time
eval_ = model1.evaluate(X_test_pad, y_test)

In [ ]:
print("Loss: {0:.5}".format(eval_[0]))
print("Accuracy: {0:.2%}".format(eval_[1]))

## Conv1D model with multiple kernel sizes:

In [13]:
from keras.layers import concatenate

In [34]:
conv = []
filter_sizes = [3,4,5]

In [35]:
embedding_layer = Embedding(input_dim=num_words,
                    output_dim=embedding_size,
                    input_length=max_len,
                    name='layer_embedding')

In [36]:
seq_input = Input(shape=(max_len,), dtype='int32')
embed_seq = embedding_layer(seq_input)

for f in filter_sizes:
    _conv = Conv1D(filters=128,kernel_size=f,activation='relu')(embed_seq)
    _pool = MaxPooling1D(5)(_conv)
    conv.append(_pool)

In [37]:
_concat = concatenate(conv, axis=1)
_conv1= Conv1D(128, 5, activation='relu')(_concat)
_pool1 = MaxPooling1D(5)(_conv1)
_pool1 = Dropout(0.5)(_pool1)
_conv2 = Conv1D(128, 5, activation='relu')(_pool1)
_pool2 = MaxPooling1D(5)(_conv2)
_flat = Flatten()(_pool2)
_flat = Dropout(0.5)(_flat)
_dense = Dense(128, activation='relu')(_flat)
preds = Dense(1, activation='sigmoid')(_dense)

In [38]:
model = Model(seq_input, preds)

In [39]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 256)          0                                            
__________________________________________________________________________________________________
embedding_3 (Embedding)         (None, 256, 50)      1000000     input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_12 (Conv1D)              (None, 254, 128)     19328       embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_13 (Conv1D)              (None, 253, 128)     25728       embedding_3[0][0]                
__________________________________________________________________________________________________
conv1d_14 

In [40]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])

In [41]:
%%time
model.fit(X_train_pad, y_train, validation_split=0.05, epochs=10, batch_size=batch_size)

Train on 23750 samples, validate on 1250 samples
Epoch 1/10
23750/23750 [==============================] - 28s 1ms/step - loss: 0.5837 - acc: 0.6356 - val_loss: 0.3801 - val_acc: 0.8480
Epoch 2/10
23750/23750 [==============================] - 4s 174us/step - loss: 0.2785 - acc: 0.8848 - val_loss: 0.3236 - val_acc: 0.8728
Epoch 3/10
23750/23750 [==============================] - 4s 174us/step - loss: 0.1923 - acc: 0.9280 - val_loss: 0.2943 - val_acc: 0.8736
Epoch 4/10
23750/23750 [==============================] - 4s 174us/step - loss: 0.1386 - acc: 0.9489 - val_loss: 0.2997 - val_acc: 0.8736
Epoch 5/10
23750/23750 [==============================] - 4s 174us/step - loss: 0.0922 - acc: 0.9681 - val_loss: 0.3669 - val_acc: 0.8720
Epoch 6/10
23750/23750 [==============================] - 4s 173us/step - loss: 0.0525 - acc: 0.9829 - val_loss: 0.5083 - val_acc: 0.8568
Epoch 7/10
23750/23750 [==============================] - 4s 174us/step - loss: 0.0279 - acc: 0.9906 - val_loss: 1.2122 - va

In [42]:
%%time
eval_ = model.evaluate(X_test_pad, y_test)

25000/25000 [==============================] - 3s 108us/step
Wall time: 2.71 s


In [43]:
print("Loss: {0:.5}".format(eval_[0]))
print("Accuracy: {0:.2%}".format(eval_[1]))

Loss: 0.87396
Accuracy: 85.02%


## Reference:

https://keras.io/layers/convolutional/